In [2]:
#installing ollama
#!pip install ollama
!curl -fsSL https://ollama.com/install.sh | sh

>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
############################################################################################# 100.0%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [21]:
#starting ollama server locally
import subprocess
import time
process = subprocess.Popen("ollama serve", shell=True)
time.sleep(5)  # Wait for 5 seconds

In [22]:
#pulling llama3 using ollama
!ollama pull llama3.1:8b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling 667b0c1932bc... 100% ▕▏ 4.9 GB                         
pulling 948af2743fc7... 100% ▕▏ 1.5 KB                         
pulling 0ba8f0e314b4... 100% ▕▏  12 KB                         
pulling 56bb8bd477a5... 100% ▕▏   96 B                         
pulling 455f34728c9b... 100% ▕▏  487 B                         
verifying sha256 digest 
writing manifest 
success 


In [5]:
#testing llama 3 is availble
!ollama list

NAME           ID              SIZE      MODIFIED               
llama3.1:8b    46e0c10c039e    4.9 GB    Less than a second ago    


In [6]:
!pip install --upgrade langchain langchain_community faiss-cpu eml_parser pypdf transformers accelerate Flask pyngrok

In [36]:
import os
import json
import torch
from langchain.llms import Ollama
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
import faiss
import json
import hashlib
import numpy as np
import email
import os
import eml_parser
from datetime import datetime
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import Ollama
from scipy.spatial.distance import cosine
from bs4 import BeautifulSoup
from langchain_community.vectorstores import FAISS
import configuration
import email
from bs4 import BeautifulSoup
import os
import tempfile
from langchain.document_loaders import PyPDFLoader
from openpyxl import load_workbook
import extractEmailData
import CheckDupliateEmails
import StoreEmails
import extractJsonUtil

In [8]:
# Load LLaMA 3 Model
llm = Ollama(model="llama3.1:8b")

<ipython-input-8-3025ddad1d65>:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.1:8b")


In [9]:
# Load embedding model
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

<ipython-input-9-0f08530b9cbf>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models

In [37]:
# Function to extract key data points dynamically
def extract_key_data(text):
    prompt = f"""
    Extract key data points from the following text:

    Text: "{text}"

    Identify fields such as {configuration.keyMetrics}
    Respond in JSON format like this:
    {{
        "sender_name": "string",
        "amount": "string",
        "expiration_date": "string",
        "date_to_transfer": "string"
    }}
    """

    response = llm.invoke(prompt)
    try:
        extracted_data = extractJsonUtil.extract_json_from_string(response)
    except json.JSONDecodeError:
        extracted_data = {}
    return extracted_data

In [38]:
# Function to extract key data points dynamically
def classify_email(text):

    prompt = f"""
You are a commercial loan servicing assistant. Your task is to analyze the provided email body and identify all service requests mentioned, classifying each into a specific category and subtype.

Email Body:
{text}

Available Request Types and Subtypes:
{configuration.requestTypesAndSubTypes}

Instructions:
1. Thoroughly review the email body and identify all instances where a service request is being made.
2. For each request, determine the most appropriate request category from the provided list.
3. If applicable, select the relevant subrequest type based on the specifics of the request.
4. Base your confidence on factors like:
    - Presence of keywords related to the request category/subtype.
    - Overall semantic similarity between the request and the category/subtype description.
    - The strength and clarity of the request expressed in the email.
5. If a single sentence or phrase implies multiple requests, prioritize the most specific and explicit request.
6. If there are overlapping or ambiguous requests, use your best judgment to classify them based on the overall context of the email.

Return your classification in the following JSON format:

{{
  "requests": [
    {{
      "request_category": "Selected Category",
      "subrequest_type": "Selected Subtype",
      "confidence_reason": "Brief explanation for your confidence level"
      "confidence score percentage": ""
    }},
    {{
      "request_category": "Selected Category",
      "subrequest_type": "Selected Subtype",
      "confidence_reason": "Brief explanation for your confidence level"
      "confidence score percentage": ""
    }},
    // ... more requests
  ]
}}
"""

    response = llm.invoke(prompt)
    #print("llm res", response)
    try:
        extracted_data = extractJsonUtil.extract_requests_json(response)
    except json.JSONDecodeError:
        extracted_data = []
    return extracted_data

In [44]:
example_texts = ["This is an example text for initialization."]
vectorstore = FAISS.from_texts(example_texts, embedding_model)

In [16]:
all_docs = vectorstore.similarity_search("", k=vectorstore.index.ntotal)
print(len(all_docs))

1


In [45]:
def process_eml_file(eml_file_path):
    """Processes an EML file and returns the results."""
    results = []

    if eml_file_path.endswith(".eml"):
        email_text = extractEmailData.extract_email_data(eml_file_path)
        email_content = email_text['subject'] + email_text['body']
        email_attachments_content = email_text['attachments']

        is_dup, source, score = CheckDupliateEmails.check_duplicate(email_content, vectorstore, embedding_model, configuration.duplicateCheckThresold)

        response_json = {}
        if is_dup:
            response_json = {
                "email_text": email_content[:100] + "...",
                "classification": "duplicate",
                "duplicate": is_dup,
            }
        else:
            classification = classify_email(email_content)
            extracted_data = extract_key_data(email_content)
            StoreEmails.store_email(email_content, vectorstore, embedding_model)
            response_json = {
                "email_text": email_content[:100] + "...",
                "classification": classification,
                "duplicate": is_dup,
                "key_data": extracted_data
            }
        results.append(response_json)

        print("resuls", json.dumps(results, indent=2))

    return results


In [46]:
process_eml_file("email.eml")

Current email embedding: [0.06368917971849442, 0.0010894684819504619, -0.011865378357470036, -0.031230291351675987, 0.030863210558891296, 0.030553920194506645, -0.020263152197003365, -0.0018225387902930379, -0.05592602118849754, -0.011899597942829132, -0.05644566938281059, 0.00956113450229168, 0.015409605577588081, 0.02048807591199875, -0.05141672119498253, -0.007854587398469448, 0.014856945723295212, -0.028597582131624222, 0.04317307844758034, 0.01562570221722126, 0.000382595113478601, 0.02962465211749077, -0.03363037109375, -0.06420730799436569, -0.010811811313033104, -0.03461159020662308, -0.004750088322907686, 0.001154324971139431, 0.014179172925651073, -0.020099367946386337, 0.020061975345015526, 0.01114443689584732, 0.005008608102798462, -0.030471816658973694, 1.7758544572643586e-06, -0.027035273611545563, 0.011670087464153767, 0.012502776458859444, -0.025437092408537865, 0.09080171585083008, -0.016279200091958046, 0.009329229593276978, -0.025286447256803513, 0.037983767688274384

[{'email_text': 'Adjustment Request - Reallocation FeesDear Loan Servicing Team,\n\nWe request an adjustment to the loa...',
  'classification': [{'request_category': 'Adjustment',
    'subrequest_type': 'Reallocation Fees',
    'confidence_reason': "The presence of keywords 'reallocate the funds' and 'reallocation fees' indicates that this is an adjustment request, specifically for reallocation fees.",
    'confidence_score_percentage': 90}],
  'duplicate': False,
  'key_data': {'account_number': '#123456',
   'amount': '$50,000',
   'effective_date': '2025-04-10'}}]

In [19]:
from flask import Flask, request, jsonify
import json
from pyngrok import ngrok

app = Flask(__name__)

@app.route('/process_email', methods=['POST'])
def process_email():
    print("into process mail")
    if 'eml_file' not in request.files:
        return jsonify({'error': 'No EML file provided'}), 400

    print("eml file exists")
    eml_file = request.files['eml_file']
    eml_file.save('temp_email.eml')  # Save temporarily
    print("saved to temp file")
    results = process_eml_file('temp_email.eml')  # Call the processing method
    print("results received ", results)
    response_data = {
        "message": "Email processed successfully",
        "results": results
    }

    return jsonify(response_data), 200

if __name__ == '__main__':
     ngrok.set_auth_token("2t6u6zBAZFZh2JGODqQ0s0MyQt8_7XXGdkuk8Mm3UhCdw73WA") # signup and get auth token from https://dashboard.ngrok.com/get-started/your-authtoken
     public_url = ngrok.connect(5000).public_url
     print(f" * Running on {public_url} (Press CTRL+C to quit)")
     app.run(host='0.0.0.0', port=5000, debug=True)

 * Running on https://f866-34-125-206-204.ngrok-free.app (Press CTRL+C to quit)
 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with stat
